In [7]:
# Imports
import itertools
import functools
import multiprocess
import numpy as np

from research.data.data          import *
from research.data.utils         import *
from core.utils.entity_type      import Entity_type
from core.search.query_paper_mag import paper_mag_multiquery
from core.search.query_info      import paper_info_mag_check_multiquery

In [8]:
# Inputs
THREADS    = 8
BATCH_SIZE = 40

# List of authors
author_name_list = ['stephen m blackburn', 'k r catchpole',
                    'brendan d mckay', 'Andrew Blakers',
                    'Brian D. O. Anderson', 'Wojciech Lipinski',
                    'Robert C. Williamson', 'G.D. Farquhar',
                    'Chennupati Jagadish', 'C. Jagadish',
                    'Susan M. Scott', 'S. M. Scott',
                    'Peter M.W. Gill', 'Philip Pettit',
                    'Craig Moritz', 'David B. Lindenmayer',
                    'David Lindenmayer', 'Anna Wierzbicka']
author_name_list = list(map(name_normalise, author_name_list))

# Additional paper ids to cache
extra_papers = []

In [9]:
# Get map from names -> author ids
name_maps = get_author_ids(author_name_list)

print('Number of author ids generated:', np.sum(list(map(len, name_maps.values()))))

Number of author ids generated: 567


In [4]:
# Generate paper ids to generate in cache
papers_to_cache = set()
for name, author_ids in name_maps.items():
    papers_to_cache.update(paper_mag_multiquery(Entity_type.AUTH, author_ids))
papers_to_cache.update(extra_papers)
papers_to_cache = list(papers_to_cache)

print('Number of paper ids generated:', len(papers_to_cache))

return statue: 403
{'Ocp-Apim-Subscription-Key': '609ec536d4fc427988d33aa4401ec4e2'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '


KeyError: 'entities'

In [ ]:
# Set up caching process for paper ids
p = multiprocess.Pool(THREADS)
    
# Set up for threads
batches = (papers_to_cache[i:i+BATCH_SIZE] for i in \
               range(0, len(papers_to_cache), BATCH_SIZE))

# Batch job with enumeration
def job(enum_input):
    batch_number, batch = enum_input
    print('Starting batch job:', batch_number, '| Size:', len(batch))
    res = paper_info_mag_check_multiquery(batch)
    print('Finishing batch job:', batch_number)
    return res

batch_res = p.map(job, list(enumerate(batches)))
paper_information = functools.reduce(lambda x, y: x + y, batch_res)